In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
df = pd.read_csv("ORCL.csv")

In [19]:
#evealuate whether the week is good or bad
def evaluateWeek(fri_to_fri, results):
    length = fri_to_fri.size
    ave = np.mean(fri_to_fri)
    st = np.std(fri_to_fri)
    if length >= 4:
        #if the average of the return of the week is more than 0 and the std is less than 1, the week is goood
        if ave > 0 and st < 1:
            results.append([length, 'Green'])
        else:
            results.append([length, 'Red'])
    else:
        results.append([length, None])
   
    return results

In [20]:
#get list of length of week and color corresponding to th week
def getGoodWeekdays(df, year):
    df_year = df.loc[df['Year'] == year].copy()
    returns = df_year['Return']
    weekdays = df_year['Weekday']
    length_year = returns.size
    count = 0
    week_start = False
    keep = None
    results = []
    for day, ret in zip(weekdays, returns):

        count += 1 
        #when the week start
        if not week_start:
            fri_to_fri = np.array([])
            week_start = True
            #append keep return from last week to this week 
            if keep:
                fri_to_fri = np.append(fri_to_fri, keep)
                keep = None
        #append return to list
        fri_to_fri = np.append(fri_to_fri, ret)
        
        #when the week end by Friday
        if week_start and day =='Friday':
            week_start = False
            results = evaluateWeek(fri_to_fri, results)
        #when the week exceed 5 days
        elif week_start and fri_to_fri.size > 5:
            fri_to_fri = fri_to_fri[:-1]
            keep = fri_to_fri[-1]
            week_start = False
            results = evaluateWeek(fri_to_fri, results)
        #when the week end at the the end of year
        elif week_start and length_year - count == 0:
            week_start = False
            results = evaluateWeek(fri_to_fri, results)
    return results

#get dataframe with weeks and corresponding color.
def getGoodWeekDf(df, year, results):
    df_year = df.loc[df['Year'] == year].copy()
    goodness = []
    week = []
    n_week = 0
    for result in results:
        n_week += 1
        for i in range(result[0]):
            goodness.append(result[1])
            if result[1] != None:
                week.append(n_week)
            else:
                week.append(np.nan)
    df_year['Week'] = week
    df_year['Goodness'] = goodness
    return df_year

In [21]:
goodness_2018 = getGoodWeekdays(df, 2018)
df_2018 = getGoodWeekDf(df, 2018, goodness_2018)

In [32]:
goodness_2017 = getGoodWeekdays(df, 2017)
df_2017 = getGoodWeekDf(df, 2017, goodness_2017)

In [23]:
def getMeanAndStdDict(df_year):
    weeks = df_year['Week']
    returns = df_year['Return']
    len_week = weeks.size
    mean_dict = {}
    std_dict = {}
    last_week = None
    arr = np.array([])
    n_week = 0
    for week, ret in zip(weeks, returns):
        
        n_week += 1
        #append retern to arr 
        if last_week == week:
            arr = np.append(arr, ret)
        #if week of last loop and current week is different and arr is not empty
        elif last_week != week and arr.size != 0:
            mean_dict[last_week] = np.mean(arr)
            std_dict[last_week] = np.std(arr)
            arr = np.array([ret])
        #first loop     
        elif arr.size == 0:
            arr = np.append(arr, ret)
        
        last_week = week

        #last loop    
        if len_week == n_week :
            mean_dict[last_week] = np.mean(arr)
            std_dict[last_week] = np.std(arr)
        
    return mean_dict, std_dict


In [24]:
mean_2018, std_2018 = getMeanAndStdDict(df_2018)

In [33]:
mean_2017, std_2017 = getMeanAndStdDict(df_2017)

In [25]:
def getWeeklyReturnDict(df_year):
    weeks = df_year['Week']
    adjs = df_year['Adj Close']
    len_week = weeks.size
    weekly_return_dict = {}
    start_price = np.nan
    last_week = None
    n_week = 0
    for week, adj in zip(weeks, adjs):
        n_week += 1
        #get end price of last week and calculate return of last week and set start price for this week
        if last_week != week and not np.isnan(start_price):
            end_price = last_price
            weekly_return_dict[last_week] = (start_price - end_price) / start_price
            start_price = adj
        #First loop
        elif  np.isnan(start_price):
            start_price = adj

        last_week = week
        last_price = adj
        #last_loop
        if len_week == n_week:
            end_price = adj
            weekly_return_dict[last_week] = (start_price - end_price) / start_price
    return weekly_return_dict



In [26]:
weekly_return_2018 = getWeeklyReturnDict(df_2018)

In [35]:
weekly_return_2017 = getWeeklyReturnDict(df_2017)

In [36]:
#change dictionary to np.array
def changeToNpArray(dictionary):
    arr = np.array([])
    for k, v in dictionary.items():
        if not np.isnan(k):
            arr = np.append(arr, v)
    return arr
#change list to np.array
def changeToNpArrayColor(color_list):
    arr = np.array([])
    for v in color_list:
        if not v[1] == None:
            arr = np.append(arr, v[1])
    return arr

In [39]:
mean_arr_2018 = changeToNpArray(mean_2018)
std_arr_2018 = changeToNpArray(std_2018)
weekly_return_arr_2018 = changeToNpArray(weekly_return_2018)
color_arr_2018 = changeToNpArrayColor(goodness_2018)

In [40]:
mean_arr_2017 = changeToNpArray(mean_2017)
std_arr_2017 = changeToNpArray(std_2017)
weekly_return_arr_2017 = changeToNpArray(weekly_return_2017)
color_arr_2017 = changeToNpArrayColor(goodness_2017)